# TensorFlow/Keras vs PyTorch — porównanie tworzenia modeli i warstw

Cel: Krótkie, praktyczne porównanie sposobu definiowania modeli w TensorFlow/Keras i w PyTorch oraz mapowanie najpopularniejszych warstw/klas. Na końcu: notatka o różnicach (także dot. liczenia gradientów) i mini‑przykłady gradientów w obu bibliotekach.




## Szybki przegląd API

- Keras (tf.keras): wysoki poziom, dwa style definiowania modeli:
  - Sequential (proste, warstwa po warstwie)
  - Functional (dowolne grafy, wiele wejść/wyjść)
- PyTorch: definiujemy klasę dziedziczącą po `nn.Module` i implementujemy metodę `forward`.
- Trening:
  - Keras: `model.compile(...); model.fit(...)`
  - PyTorch: pętla treningowa: `zero_grad() → forward → loss → backward() → step()`


## Mapowanie najpopularniejszych warstw i klas

- Gęste
  - Keras: `layers.Dense(units, activation=...)`
  - PyTorch: `nn.Linear(in_features, out_features)` + aktywacja osobno (np. `nn.ReLU()`)
- Konwolucje 2D
  - Keras: `layers.Conv2D(filters, kernel_size, strides, padding)`
  - PyTorch: `nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)`
- Spłaszczanie
  - Keras: `layers.Flatten()`
  - PyTorch: `nn.Flatten()` lub `x.view(batch, -1)`
- Dropout
  - Keras: `layers.Dropout(rate)`
  - PyTorch: `nn.Dropout(p)`
- Batch Normalization (2D)
  - Keras: `layers.BatchNormalization()`
  - PyTorch: `nn.BatchNorm2d(num_features)`
- Max/Average Pooling 2D
  - Keras: `layers.MaxPooling2D(pool_size)`, `layers.AveragePooling2D(pool_size)`
  - PyTorch: `nn.MaxPool2d(kernel_size)`, `nn.AvgPool2d(kernel_size)`
- Global Average Pooling
  - Keras: `layers.GlobalAveragePooling2D()`
  - PyTorch: `nn.AdaptiveAvgPool2d((1, 1))` + `Flatten`
- Embedding
  - Keras: `layers.Embedding(input_dim, output_dim)`
  - PyTorch: `nn.Embedding(num_embeddings, embedding_dim)`
- RNN/LSTM/GRU
  - Keras: `layers.SimpleRNN`, `layers.LSTM`, `layers.GRU`
  - PyTorch: `nn.RNN`, `nn.LSTM`, `nn.GRU`
- Aktywacje
  - Keras: `activation='relu'` lub osobno `layers.ReLU()`
  - PyTorch: `nn.ReLU()`, `nn.Sigmoid()`, `nn.Softmax(dim=...)` itd.


## Minimalny przykład: MLP w Keras (Sequential) vs PyTorch (nn.Module) vs fasti (PyTorch)

Poniżej: ten sam pomysł na mały klasyfikator (dane sztuczne). Najpierw Keras, potem PyTorch.


In [4]:
# KERAS: prosty MLP (Sequential) i szybki trening na danych sztucznych
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
# PYTORCH: prosty MLP (nn.Module) i szybki trening na danych sztucznych
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [7]:
# KERAS: prosty MLP (Sequential) i szybki trening na danych sztucznych
np.random.seed(0)

# Dane sztuczne: 1000 próbek, 20 cech, 3 klasy
X = np.random.randn(1000, 20).astype("float32")
y = np.random.randint(0, 3, size=(1000,)).astype("int32")

model_keras = keras.Sequential(
    [
        layers.Input(shape=(20,)),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(3, activation="softmax"),
    ]
)
model_keras.compile(
    optimizer=keras.optimizers.Adam(1e-2),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
history = model_keras.fit(X, y, epochs=5, batch_size=32, verbose=0)
print("Keras — ostatnia epoka accuracy:", history.history["accuracy"][-1])

Keras — ostatnia epoka accuracy: 0.41200000047683716


Teraz PyTorch: definiujemy klasę `nn.Module` i pętlę uczącą.


In [ ]:
# PYTORCH: prosty MLP (nn.Module) i szybki trening na danych sztucznych
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [20]:
# PYTORCH: prosty MLP (nn.Module) i szybki trening na danych sztucznych
torch.manual_seed(0)


class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 32), nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(32, 3)
        )

    def forward(self, x):
        return self.net(x)


# Dane sztuczne: 1000 próbek, 20 cech, 3 klasy
X = np.random.randn(1000, 20).astype("float32")
y = np.random.randint(0, 3, size=(1000,)).astype("int32")
X_t = torch.tensor(X)
y_t = torch.tensor(y, dtype=torch.long)
train_loader = DataLoader(TensorDataset(X_t, y_t), batch_size=32, shuffle=True)
model_torch = MLP()
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model_torch.parameters(), lr=1e-2)
model_torch.train()
for epoch in range(5):
    for xb, yb in train_loader:
        optim.zero_grad()
        logits = model_torch(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optim.step()

# Prosty accuracy na całym zbiorze
model_torch.eval()
with torch.no_grad():
    preds = model_torch(X_t).argmax(dim=1)
acc = (preds == y_t).float().mean().item()
print("PyTorch — accuracy:", acc)

PyTorch — accuracy: 0.5350000262260437


In [21]:
# dodam tez fastai - taki keras dla PyTorch
from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.metrics import accuracy
from fastai.optimizer import Adam

In [23]:
# Dane sztuczne
np.random.seed(0)
X = np.random.randn(1000, 20).astype("float32")
y = np.random.randint(0, 3, size=(1000,)).astype("int64")
X_t = torch.tensor(X)
y_t = torch.tensor(y)


# Model taki jak u Ciebie
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 32), nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(32, 3)
        )

    def forward(self, x):
        return self.net(x)


# Datasety fastai
train_ds = TensorDataset(X_t, y_t)
dls = DataLoaders.from_dsets(train_ds, train_ds, bs=32, shuffle=True)
# Learner z fastai Adam
learn = Learner(
    dls, MLP(), loss_func=nn.CrossEntropyLoss(), opt_func=Adam, metrics=accuracy
)
learn.fit(5, lr=1e-2)
acc = learn.validate()[1]
print("fastai — accuracy:", acc)

[0, 1.1273056268692017, 1.0761957168579102, 0.4180000126361847, '00:00']
[1, 1.0985183715820312, 1.0573548078536987, 0.4449999928474426, '00:00']
[2, 1.0864070653915405, 1.0414248704910278, 0.46700000762939453, '00:00']
[3, 1.0765001773834229, 1.0227164030075073, 0.4740000069141388, '00:00']
[4, 1.0633543729782104, 1.007742166519165, 0.5149999856948853, '00:00']
fastai — accuracy: 0.5149999856948853


## Functional API (Keras) vs dowolne grafy w PyTorch

- Functional Keras pozwala łączyć warstwy w graf z wieloma wejściami/wyjściami.
- PyTorch daje pełną swobodę w `forward` (instrukcje warunkowe, pętle, gałęzie).

Krótki przykład Functional w Keras:


In [24]:
from tensorflow.keras import Input, Model

inp = Input(shape=(20,))
h = layers.Dense(32, activation="relu")(inp)
h = layers.Dropout(0.1)(h)
out = layers.Dense(3, activation="softmax")(h)
func_model = Model(inputs=inp, outputs=out)
func_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771 (3.01 KB)

 Trainable params: 771 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

W PyTorch ekwiwalent jest po prostu arbitralną logiką w `forward`:


In [26]:
class BranchyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(20, 32)
        self.fc2 = nn.Linear(32, 3)
        self.dropout = nn.Dropout(0.1)
        self.act = nn.ReLU()

    def forward(self, x):
        h = self.act(self.fc1(x))
        if self.training:
            h = self.dropout(h)
        return self.fc2(h)


m = BranchyNet()
print(m)

BranchyNet(
  (fc1): Linear(in_features=20, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=3, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (act): ReLU()
)


## Uwaga o wymiarach (NCHW vs NHWC)

N – batch size

H – height

W – width

C – channels

- Keras/TensorFlow (domyślnie): dane obrazowe często w układzie NHWC (batch, height, width, channels).
- PyTorch: standardowo NCHW (batch, channels, height, width). 
- Wpływa to na parametry warstw (np. `in_channels`) i konieczność permutacji wymiarów przy konwersji między frameworkami.


## Różnice: TensorFlow vs Keras (i liczenie gradientów)

- Keras to wysoki poziom API; w TF 2.x zazwyczaj używamy `tf.keras` (Keras z silnikiem TF).
- Keras (wysoki poziom): `compile`/`fit`/`evaluate`/`predict` — wygoda i gotowe pętle.
- TensorFlow niski poziom: `tf.GradientTape` do ręcznego liczenia gradientów i pełnej kontroli.
- PyTorch: jeden spójny, niski poziom; automatyczne różniczkowanie przez Autograd, pętle treningowe piszemy sami.
- Gradienty:
  - TensorFlow: w kontekście `tf.GradientTape()` śledzimy operacje i liczymy `tape.gradient(loss, params)`.
  - PyTorch: tensory z `requires_grad=True`, wołamy `loss.backward()`, gradienty trafiają do `param.grad`.
- Różnice praktyczne:
  - Keras ukrywa sporo szczegółów (łatwy start). PyTorch wymaga więcej kodu, ale daje pełną elastyczność w `forward` i pętli treningowej.


## Mini‑przykłady gradientów

Pokażemy pochodną funkcji f(x) = x^2 w jednym punkcie w obu bibliotekach.


In [27]:
# TensorFlow: GradientTape
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y = x**2
(dy_dx,) = tape.gradient(y, [x])
print("TF — dy/dx dla x=3:", dy_dx.numpy())

TF — dy/dx dla x=3: 6.0


In [28]:
# PyTorch: Autograd
t = torch.tensor(3.0, requires_grad=True)
y = t**2
y.backward()
print("PyTorch — dy/dx dla x=3:", t.grad.item())

PyTorch — dy/dx dla x=3: 6.0


## Krótkie podsumowanie

- Definicja modeli:
  - Keras: `Sequential` (najszybciej) lub `Functional` (elastycznie, wieloweściowy graf).
  - PyTorch: klasa `nn.Module` + `forward` (pełna kontrola przepływu danych).
- Warstwy mają bezpośrednie odpowiedniki (lista na górze), różnią się nazwami i detalami argumentów.
- Gradienty: `tf.GradientTape` vs Autograd/`backward()`. Keras na wysokim poziomie ukrywa szczegóły w `fit()`.

Gotowe — możesz skopiować i adaptować pokazane szablony do własnych projektów.